In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor 
from pycor import utils, korlang

stopwatch = utils.StopWatch()


model_size = 7000
outputpath = "../../output/" + str(model_size) + "/"
model_path = outputpath + "model/"

pycor.loadmodel(model_path)



Init Trainer
Init DocResolver
Loading Model from ../../output/7000/model/
Load  ../../output/7000/model//heads.csv   소요시간: 0.819
Load  ../../output/7000/model//tails.csv   소요시간: 0.011


In [2]:
def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = []
files.extend( listfiles('../../data/wiki2') )
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../samples/news') )
files.extend( listfiles('../../data/news') )
files.extend( listfiles('../../data/blotter') )
files.extend( listfiles('../../data/zdnet') )
files.extend( listfiles('../../data/ciobiz') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")

3959 files


In [3]:

yeTags = set(['EFN'])

def gettail(word):
    if word.bestpair:
#         return word.bestpair.tags
        return word.bestpair.tail.text
    return 'X'

def gettag(word):
    if word.bestpair:
        return word.bestpair.tags
    return 'X'

def gethead(word):
    if word.bestpair:
        return word.bestpair.head.text
    return 'X'


yheadsMap = {}
ytailsMap = {}
ytagsMap = {}

def handleHaed(index, word, words):
    yset = yheadsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        yheadsMap[word.bestpair.head] = yset

    if index > 1:
        yset.append( [gethead(words[index-1]),gethead(words[index-2])])
    else:
        yset.append( [gethead(words[index-1])] )
        
def handleTag(index, word, words):
    yset = ytagsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytagsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettag(words[index-1]),gettag(words[index-2])])
    else:
        yset.append( [gettag(words[index-1])] )
                
def handleTail(index, word, words):
    yset = ytailsMap.get(word.bestpair.head)
    if yset is None:
        yset = []
        ytailsMap[word.bestpair.head] = yset
    if index > 1:
        yset.append( [gettail(words[index-1]),gettail(words[index-2])])
    else:
        yset.append( [gettail(words[index-1])] )
        
def analyzeYongEon(file):
    sentence_array, words_array = pycor.readfile(file)
    
    for words in words_array:
        for index, word in enumerate(words):
            if word.bestpair and len(yeTags & set(word.bestpair.tags))>0 and word.text.endswith('다'):
                handleHaed(index, word, words)
                handleTail(index, word, words)
                handleTag(index, word, words)


def writeFile(path, yMap):
    with open(path, 'w', encoding='utf-8') as file :
        writer = csv.writer(file)
        for head, tags in yMap.items():
            headText = head.text
            for tag in tags:
                row = [headText]
                row.extend(tag)
                writer.writerow(row)
        file.close()
                         
stopwatch.start()

docsize = 70

for file in files[:docsize]:
    if file.endswith(".txt"):
        analyzeYongEon(file)

print("Loading ",docsize," Docs: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")

Loading  70  Docs:  2s.119ms. ( 2,118 ms.)


In [4]:
# yheadsMap 
# ytailsMap  
# ytagsMap  

def fill(ylist, ymap):
    for head, tags in ymap.items():
        headText = head.text
        for tag in tags:
            row = [headText]
            row.extend(tag)
            ylist.append(row) 
        
tailsList = []
tagsList = []

fill(tailsList,ytailsMap)
fill(tagsList,ytagsMap)

josaYMap = {}
matrix = []

for index in range(len(tailsList)):
    tail = tailsList[index]
    tag = tagsList[index]
    
    text = tail[0]
    row = [text]
    row.extend(tail[1:])
    
    josaKey = ''
    for tags in tag[1:]:
        tagsStr = "+".join(tags)
        row.append(tagsStr)
        josaKey += "::"
        for t in tags:
            if t.startswith("J"):
                josaKey += "+" + t
    
    ymap = josaYMap.get(josaKey)
    if ymap is None:
        ymap = {}
        josaYMap[josaKey] = ymap
    
    count = ymap.get(text)
    if count is None:
        count = 0
    
    count += 1
    ymap[text] = count
    
    matrix.append(row)

    
def writeTailTagsFile(path, tailTagsList):
    with open(path, 'w', encoding='utf-8') as file :
        writer = csv.writer(file)
        for row in tailTagsList:
            writer.writerow(row)
                
        file.close()
        
outputpath = "../../output/" + str(docsize) + "/"

writeFile(outputpath+'/yongeon-heads.csv',yheadsMap)
# writeFile(outputpath+'/yongeon-tails.csv',ytailsMap)
# writeFile(outputpath+'/yongeon-tags.csv',ytagsMap)


writeTailTagsFile(outputpath+'/yongeon-tails-tags.csv',matrix)

print(len(matrix))

print(len(josaYMap))

1580
160


In [5]:
from pycor.res import CollocationResolver
import pycor.speechmodel as sm 

resolver = CollocationResolver()
pycor.addresolver(resolver)

def gettext(word, islast=False):
    if type(word) is sm.Sentence :
        if(word.senttype == sm.SENTENCE_TYPE_EQUIV):
            return word.text()
        else:
            return word.text()
    else:
        if islast and word.bestpair:
            return word.bestpair.head.text
        else:
            return word.text
            

sentence_array, words_array = pycor.readfile('../samples/docs/economics1.txt')

ngrams = {}

for sentence in sentence_array:
    for index in range(1, len(sentence.words)-1):
        first = gettext(sentence.words[index-1])
        second = gettext(sentence.words[index])
        second2 = gettext(sentence.words[index], True)
        third = gettext(sentence.words[index+1], True)
        
        bigram = ' '.join([first,second2])
        trigram = ' '.join([first,second,third])
        
        bigramCnt = ngrams.get(bigram)
        if bigramCnt:
            bigramCnt = bigramCnt +1
        else:
            bigramCnt = 1
        ngrams[bigram] = bigramCnt
        
        trigramCnt = ngrams.get(trigram)
        if trigramCnt:
            trigramCnt = bigramCnt +1
        else:
            trigramCnt = 1
        ngrams[trigram] = trigramCnt
#         ngrams[bigram] = [sentence.words[index-1], sentence.words[index]]
#         ngrams[trigram] = [sentence.words[index-1], sentence.words[index],sentence.words[index+1]]
#         print(first, second , third )

# for key, cnt in ngrams.items():
#     if cnt > 1:
#         print(key,cnt)

# for colloc in pycor.getmodel().collocations.values():
#     print(colloc, colloc.frequency)

In [6]:
for file in files[:400]:
    if file.endswith(".txt"):
        pycor.readfile(file)
        

print( len(pycor.getmodel().collocations))
for colloc in pycor.getmodel().collocations.values():
    if colloc.frequency > 3:
        print(colloc, colloc.frequency)

13760
통화 정책 10
화폐 정책 6
화폐의 공급 6
수 있다 522
할 수 188
할 수 있다 90
시장 기구 4
자유 경쟁 5
함께 생산 4
노동의 대가 8
노동의 대가로서 임금 4
대가로서 임금 6
대가로서 임금이 지급되 6
임금이 지급되 6
등과 함 9
마르크스는 노동 4
자본주의 경제 15
상품의 가치 12
근로자라 할 4
근로자라 할 수 4
수 없 197
OECD 국가 5
노동 시간이 4
타인을 위한 120
예를 들어 83
자원은 인간 4
필요로 하 24
따라서 자원 4
등과 같이 10
발전에 따라 6
인간의 요구 6
자원을 이해하 4
자원을 이해하는  4
이해하는  8
이해하는 데 있어 6
데 있어 11
수 있 705
이해할 수 6
이해할 수 있다 6
되는 것 40
자원을 개발 4
자원을 개발하고 이용하 4
개발하고 이용하 4
자원의 개발 10
있어서는 자원 4
문제가 되 10
지역 국가  6
그 자체 37
자원의 분포 6
문제가 되는 것 4
막대한 소비 4
개개의 자원 4
그래서 자원 4
치열한 경합 4
그 결 31
광물자원의 개발 5
지역에 따라 12
따라 그 8
가까운 곳 6
저항이 따르 4
사회적 저항 5
극복하는 문제 4
한 쪽 9
남벌 濫伐  4
있지 않 29
자원에 대한 5
무한하게 커지 4
재생할 수 4
재생할 수 있 4
수 있는 자원 7
있는 자원 7
과정에 있다 5
인공림의 조성 4
등으로 지력 4
가고 있다 4
물질과 동식물 4
물질과 동식물 인류 4
동식물 인류 4
지난 시대 4
남긴 흔적 4
경제 실험 4
양자가 다른 5
양자가 다른 경우 5
다른 경우 7
이라 한 14
대한민국 민법 8
채무자의 채무불이행 4
두 회사 5
경우 공공기관 4
제공하고 있다 7
지급할 것 14
피보험자의 생명 4
관한 보험사고 4
관한 보험사고가 생길 4
보험사고가 생길 4
보험사고가 생길 경우 4
생길 경우 6
보험금액을 지급하 8
사망을 보험사고 9
사망을 보험사고로 하 7
보험사고로 하 15
그 사람 5
보험사고로 하는 보험계약 6
하

In [7]:
# pycor.readfile('../samples/news/new1_hani.txt')

# for colloc in pycor.getmodel().collocations.values():
#     print(colloc, colloc.frequency)